In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE','proj_demo.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from app_demo.models import Department,Employee,Customer,Product,Sales,SalesItem
import datetime
import calendar
from django.db.models import Max,Min
import pandas as pd
import plotly.express as px

In [ ]:
def get_period_tree(dt_min,dt_max):
    dict_period = {}
    list_dict_tree = []
    for y in range(dt_max.year,dt_min.year-1,-1):
        dict_node = {}
        di_detail = {}
        di_detail['from_date'] = datetime.date(y,1,1)
        di_detail['to_date'] = datetime.date(y,12,31)
        dict_period[y] = di_detail
        list_dict_nodes = []
        
        for m in range(di_detail['to_date'].month,di_detail['from_date'].month-1,-1):
            dict_child_node = {}
            res = calendar.monthrange(y,m)
            di_detail = {}
            dt_start = datetime.date(y,m,1)
            name = dt_start.strftime('%b-%Y')
            di_detail['from_date'] = dt_start
            di_detail['to_date'] =  datetime.date(y,m,res[1])
            dict_period[name] = di_detail
            
            dict_child_node['name'] = name
            dict_child_node['icon'] = 'fa fa-bars'
            dict_child_node['children'] = []
            list_dict_nodes.append(dict_child_node)
            
        dict_node['name'] = str(y)
        dict_node['icon'] = 'far fa-calendar'
        dict_node['children'] = list_dict_nodes
        list_dict_tree.append(dict_node)
        
    return dict_period,list_dict_tree

In [ ]:
dict_range = Sales.objects.aggregate(dt_min_sales=Min("date"),dt_max_sales=Max("date"))
dt_min = dict_range['dt_min_sales']
dt_max = dict_range['dt_max_sales']
#dict_period,list_dict_tree = get_period_tree(dt_min,dt_max)
dict_range


In [ ]:
from_date = dict_period['Dec-2021']['from_date']
to_date = dict_period['Dec-2021']['to_date']
qs =  Sales.objects.filter(date__gte=from_date,date__lte=to_date)
len(qs)

In [ ]:
cust = Customer.objects.get(id='C012')
cust_sales = Sales.objects.filter(customer=cust).order_by('date')
df = pd.DataFrame.from_records(cust_sales.values())
#df
#fig = px.line(df, x='date', y="net_amount")
#fig.show()
df['date'] = df['date'].astype('datetime64')
df['year'] = df['date'].dt.year
df['year'] = df['year'].astype('str')
df_summary =  df.groupby('year').agg({'net_amount':['sum']})
df_summary.columns = ['total_sales']
df_summary = df_summary.reset_index()
df_summary
fig = px.pie(df_summary, values='total_sales', names="year")
fig.show()


In [ ]:
import app_demo.data.context_menu as context_menu

cmenu_provider1 = context_menu.ContextMenuProvider.get_instance()
cmenu_provider2 = context_menu.ContextMenuProvider.get_instance()
cmenu_provider2._get_context_menu('sales_right_click')


In [ ]:
cmenu_provider1._get_context_menu('sales_right_click')

In [ ]:
import app_demo.data.context_menu as context_menu
cmenu_provider4 = context_menu.ContextMenuProvider()
cmenu_provider4._get_context_menu('sales_right_click')